# 機械学習の実装 3 （ハイパーパラメータ調整）

前章までにも何度か紹介していたハイパーパラメータの調整方法について学びます。  
本章では、モデルの予測精度を向上させるための重要な概念になるハイパーパラメータについて基礎的な概要から、具体的な調整方法と調整後の検証方法までの一連手順を解説します。  

## 本章の構成

- ハイパーパラメータの概要と交差検証
- ハイパーパラメータの調整方法


## ハイパーパラメータの概要と交差検証


### ハイパーパラメータとは
パラメータはモデルの学習実行後に獲得される値を指していました。  

ハイパーパラメータは各アルゴリズムに付随して、アルゴリズムの挙動を制御するための値です。  
**学習の実行前**に設定値を調整することでモデルの性能向上や、過学習を抑制することができます。  

### ホールドアウト法
前章までは与えられたデータセットを学習用データセット・テスト用データセットを 2 分割しましたが、実際の開発時にはモデルの性能評価をより適切にするためにデータを 3 分割してモデルを評価する必要があります。  

| データ名称 | 使用目的                                         |
| ------ | -------------------------------------------- |
| 学習用データセット (train) | モデルを学習させるためのデータセット                       |
| 検証用データセット (validation) | ハイパーパラメータの調整が適切なのか検証するためのデータセット |
| テスト用データセット (test)| 学習済みモデルの性能を評価するためのデータセット     |

学習用データセットと検証用データセットは学習段階で用いられ、テスト用データセットはモデルの予測精度の確認のためにのみ使用するということを抑えておきましょう。  

しかし、十分なデータ量が用意できない場合には 3 分割すると偏りが生じて適切な学習・検証が行われない可能性があります。  
そのようなデータの偏りを回避する方法として**K-分割交差検証（ K-fold cross-validation ）**があります。  

### K-分割交差検証（ K-fold cross-validation ）

K-分割交差検証は 3 つのステップから構成されており、視覚的に確認すると分かり易いため、図で解説していきます。   
前提として、K-分割交差検証は学習用データセットと検証用データセットの分割に用いることが多いです。そのため、下記の図ではテスト用データセットは既に別途分割している事とします。

まず第 1 ステップとして、データセットを k 個に分割します。  
下記の例では分割数 k を 5 にしています。


![Cross Validation1](http://drive.google.com/uc?export=view&id=1fh67YiQ5cVLnR_dKz2mMokHCaBd-Yz0w)

第 2 ステップとして、分割したデータの 1 つを検証用データセットとし、残りを学習用データセットとして学習を実行します。  

ここで重要なポイントとして 1 回で学習を終わらせず、計 k 回の学習を行います。  
その際、既に検証用データセットに使ったデータを次は学習用データセットとして使用し、新たに検証用データセットを選択します。    

![Cross Validation2](http://drive.google.com/uc?export=view&id=1HIBW4SkXnlebY47Y2geCS4kVXcKN0kyV)


第 3 ステップとして、各検証の結果を平均して最終的な検証結果とします。  
このようにすれば、データに偏りなくハイパーパラメータのチューニングを行うことができます。  

## ハイパーパラメータの調整方法

ハイパーパラメータの概要と検証方法が分かったので、続いて具体的な調整方法を見て行きましょう。  
調整方法については代表的な 4 つの方法を紹介します。  

アルゴリズムには決定木を使用し、それぞれの方法でハイパーパラメータ調整を行っていきます。

- 手動での調整  
- **グリッドサーチ （Grid Search）**  
- **ランダムサーチ （Random Search）**  
- **ベイズ最適化 （Bayesian Optimization）**  

### 手動での調整

まずは手動でハイパーパラメータの調整を行い、予測精度にどのような変化があるのかを確認しましょう。    

In [ ]:
# 必要なモジュールをインポート
import numpy as np
import pandas as pd

今回は scikit-learn に準備されている、乳がんに関するデータセットを使用します。  
陰性か陽性の 2 つの値が目標値にある、2 値分類の問題設定になります。  

In [ ]:
# 乳がんに関するデータセットの読み込み
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()

In [ ]:
t = dataset.target
x = dataset.data

In [ ]:
x.shape

In [ ]:
t.shape

まずは先ほど紹介した通りデータを学習用データセット・検証用データセット・テスト用データセットの 3 つに分割します。    
以下の手順で分割すると理解しておきましょう。  

- 与えれたデータを「テスト用データセット：その他＝ 20 ： 80 」に分割
- 「その他」のデータを「検証用データセット：学習用データセット＝ 30 ： 70 」に分割

In [ ]:
# テスト用データセット：その他＝ 20% ： 80%
from sklearn.model_selection import train_test_split
x_train_val, x_test, t_train_val, t_test = train_test_split(x, t, test_size=0.2, random_state=1)

In [ ]:
# 検証用データセット：学習用データセット＝ 30 ： 70
x_train, x_val, t_train, t_val = train_test_split(x_train_val, t_train_val, test_size=0.3, random_state=1)

分割処理の後に念のためサイズを確認するよう癖づけておきましょう。

In [ ]:
x_train.shape

In [ ]:
x_val.shape

In [ ]:
x_test.shape

データセットの準備が整ったので決定木の実装を行いましょう。  
ハイパーパラメータの調整は行わずに、デフォルトで設定されている値を使用して、学習を行い、予測精度を確認します。  

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0) # 再現性の確保

In [ ]:
model.fit(x_train, t_train)

In [ ]:
print('train : ', model.score(x_train, t_train))
print('validation : ', model.score(x_val, t_val))

訓練用データセットに対して 100%、検証用データセットに対して 92.7% の予測精度が確認できました。  
学習用データセットに対しての予測精度が高く、検証用データセットに対しては予測精度が低いという、過学習の傾向があることがわかります。  

過学習を抑制するハイパーパラメータを調整を行い、再度モデルの学習を行いましょう。  
`DecisionTreeClassifier()` メソッドの引数にハイパーパラメータの設定を記述します。  

In [ ]:
# ハイパーパラメータを設定して、モデルの定義
model = DecisionTreeClassifier(max_depth=10, min_samples_split=30, random_state=0)

In [ ]:
model.fit(x_train, t_train)

In [ ]:
print('train : ', model.score(x_train, t_train))
print('validation : ', model.score(x_val, t_val))

ハイパーパラメータの調整によって先ほどとは異なった結果が得られ、検証用データセットに対して 92.7% → 95.6% といった予測精度の向上が確認できました。  
テスト用データセットに対しても予測精度を検証してみましょう。

In [ ]:
print('test : ', model.score(x_test, t_test))

### グリッドサーチ

先程の例では手動で適当にハイパーパラメータの値を決めました。  
しかし、適当に入れた値が常に最適なハイパーパラメータである可能性は低いと言えるでしょう。最適なハイパーパラメータを獲得するにはある程度の探索（試行錯誤）を行う必要があります。    

効率的に最適なハイパーパラメータを探索する方法はいくつかあります。  
その内の 1 つがグリッドサーチになります。    

グリッドサーチはまず、ハイパーパラメータを探索する範囲を決めます。例えば下記の図のように決定木の `max_depth` と `max_leaf_nodes` の値を調整したい場合、5、10、15、20、25 のように範囲をそれぞれ決めます。（範囲の指定に特に決まりはありません。）  
この場合のハイパーパラメータの組み合わせは 5 x 5 = 25 個になります。この 25 個のハイパーパラメータの組み合わせ全てを使用して、学習・検証を行います。そして、その結果から予測精度が最も高いハイパーパラメータを採用します。  
 
しかし、グリッドサーチにはデメリットも存在します。  
実装方法を確認する前に整理しておきましょう。

- メリット：指定した範囲を全て網羅するため、漏れがなくハイパーパラメータの探索を行うことができる
- デメリット：場合によっては、数十～数百パターンの組合せを計算するため学習に時間を要する

![グリッドサーチ](http://drive.google.com/uc?export=view&id=1Yj_ruzw3WoFC7fgGusTGz7MzoiZQVejC)


グリッドサーチの概要が理解できたところで、実装を行います。  
グリッドサーチの実装は scikit-learn の中で準備されている `GridSearchCV` クラスを用いて実装を行います。  

In [ ]:
# GridSearchCV クラスのインポート
from sklearn.model_selection import GridSearchCV

`GridSearchCV` クラスの使用には下記の 3 つを準備する必要があります。  

- `estimator` ：学習に使用するモデル  
- `param_grid` ：ハイパーパラメータを探索する範囲  
- `cv` ：K-分割交差検証の k の値  

まずは `estimator` を定義します。 `estimator` はこれまでモデルの定義で定義していたモデルを指します。

In [ ]:
# 学習に使用するアルゴリズムの定義
estimator = DecisionTreeClassifier(random_state=0)

ハイパーパラメータの探索する範囲を指定します。  
範囲の指定は、辞書型で調整するハイパーパラメータの名前を Key に、 リスト型の探索する範囲を Value に格納します。  
調整するハイパーパラメータの名前を間違うとエラーになるため、確認して名前を記述するようにしましょう。  

In [ ]:
# ハイパーパラメータを探索する範囲の定義
param_grid = [{
    'max_depth':[3, 20, 50] , 
    'min_samples_split':[3, 20, 30]}]

In [ ]:
# 分割数 k の値の定義
cv = 5

`Grid SearchCV` では K-分割交差検証が行われます。  
そのため、学習用データセットと検証用データセットに分割する前のデータセットである `x_train_val` と `t_train_val` を使用します。  
`return_train_score=False` を設定することで学習に対する予測精度の検証が行われません。もし、検証を行う際には `True` に変更します。`False` にするメリットは計算コストを抑えることにあります。

In [ ]:
# GridSearchCV クラスを用いたモデルの定義
tuned_model = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=cv, return_train_score=False)

`GridSearchCV` クラスでも、これまでと同様に `fit()` メソッドでモデルの学習を行うことができます。

In [ ]:
# モデルの学習＆検証
tuned_model.fit(x_train_val, t_train_val)

学習結果は `cv_results_` で確認することができます。  
辞書型で格納されているため、Pandas のデータフレーム型に変換して確認すると見やすく表示することができます。  

In [ ]:
# 検証結果の確認
pd.DataFrame(tuned_model.cv_results_).T

ハイパーパラメータの種類が 2 つで、各 3 個ずつ値を指定したので 3 × 3 = 9 パターンの計算が行われています。  
また、k を 5 としたので 5 種類の結果( `split0_test_score` ~ `split4_test_score`  )が出力されています。  

 それぞれの項目の概要は下記になります。  
 
 | 項目名                  | 説明                                                  |
| :---------------------- | ----------------------------------------------------- |
| mean_fit_time           | 学習時間の平均                                        |
| std_fit_time            | 学習時間の標準偏差                                    |
| mean_score_time         | 検証時間の平均                                        |
| std_score_time          | 検証時間の標準偏差                                    |
| param_max_depth         | max_depth の値                                        |
| param_min_samples_split | min_samples_split の値                                |
| params                  | 調整しているハイパーパラメータの値                    |
| split0_test_score       | 交差検証 1 回目の検証用データセットに対しての予測精度 |
| split1_test_score       | 交差検証 2 回目の検証用データセットに対しての予測精度 |
| split2_test_score       | 交差検証 3 回目の検証用データセットに対しての予測精度 |
| split3_test_score       | 交差検証 4 回目の検証用データセットに対しての予測精度 |
| split4_test_score       | 交差検証 5 回目の検証用データセットに対しての予測精度 |
| mean_test_score         | 検証用データセットに対しての予測精度の平均            |
| std_test_score          | 検証用データセットに対しての予測精度の標準偏差        |
| rank_test_score         | 検証用データセットに対しての予測精度の順位            |
 
`mean_test_score` の値を確認するとそのモデルの予測精度の確認ができます。基本的にはこの値を確認し、どのハイパーパラメータが効果が強いのかを確認します。   
 
 
その後、結果を参照して先ほどより狭い範囲でハイパーパラメータを調整します。  
これを何度か繰り返すことで徐々に予測精度が高くなるハイパーパラメータへと近づけて行きます。    

In [ ]:
estimator = DecisionTreeClassifier(random_state=0)

In [ ]:
param_grid = [
    {'max_depth':[5, 10, 15] , 'min_samples_split':[10, 12, 15]}
]

In [ ]:
cv = 5

In [ ]:
# モデルの定義
tuned_model = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=cv, return_train_score=False)

In [ ]:
# モデルの学習＆検証
tuned_model.fit(x_train_val, t_train_val)

In [ ]:
# 学習結果の確認
pd.DataFrame(tuned_model.cv_results_).T

グリッドサーチ 2 回目の結果を確認できました。  
このように、最初はある程度大きな幅を持ってグリッドサーチを行い、徐々に範囲を狭めてより予測精度の高いハイパーパラメータを探していきます。  

最後にテストデータを用いて、グリッドサーチで学習させたモデルの予測精度を確認しましょう。

In [ ]:
# 最も予測精度の高かったハイパーパラメータの確認
tuned_model.best_params_

`best_estimator_` で最も検証用データセットに対しての予測精度が最も高かったハイパーパラメータで学習したモデルを取得することができます。  
取得したモデルを新たに `model` という変数に格納します。

In [ ]:
# 最も予測精度の高かったモデルの引き継ぎ
model = tuned_model.best_estimator_

In [ ]:
# モデルの検証
print(model.score(x_train_val, t_train_val))
print(model.score(x_test, t_test))

先程手動でハイパーパラメータの調整を行ったモデルのテスト用データセットに対する予測精度より精度が向上していることが確認できます。

### ランダムサーチ

グリッドサーチの 1 つの欠点として、指定した全てのハイパーパラメータを探索する点にあります。  
全てを探索するということはそれだけ計算コストが増えることを意味します。  

そこで、ランダムサーチは指定した範囲のハイパーパラメータをランダムに抽出し、学習・検証を行います。  
この方法により、広い範囲を探索することがより効率的に可能になります。  

しかし、もちろん全てのハイパーパラメータを探索するわけではないため、そのハイパーパラメータが最適化は判断が難しい点がランダムサーチの欠点と言えるでしょう。  

文献の中では、経験的にグリッドサーチと比較して、ランダムサーチの方が効率的にハイパーパラメータを探索することができるケースもあると説明しているものもあります。ランダムサーチの詳細は[こちら](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search)を参照してください。

実装方法を確認しましょう。  

In [ ]:
# RandomizedSearchCV クラスのインポート
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# 学習に使用するアルゴリズム
estimator = DecisionTreeClassifier(random_state=0)

ハイパーパラメータを探索する範囲の指定します。   
指定方法はグリッドサーチと同様になります。今回はランダムサーチの挙動を確認するために、範囲を少し広げて指定します。  
範囲の指定に `range(開始値, 終了値, ステップ)` を使用します。例えば `range(1, 10, 2)` の場合、 1 から 10 までの値を 2 刻みで獲得できます。その値を `list()` でリスト化しています。

In [ ]:
list(range(1, 10, 2))

In [ ]:
# ハイパーパラメータを探索する範囲の指定
param_distributions ={'max_depth':list(range(5, 100, 2)) , 'min_samples_split':list(range(2, 50, 1))}

ランダムサーチはグリッドサーチ異なり、指定した範囲のハイパーパラメータをランダムに抽出し学習を行うため、何回学習を試行するかの回数を指定する必要があります。

In [ ]:
# 試行回数の指定
n_iter = 100

`RandomizedSearchCV` クラスでも k-分割交差検証が行われるため、 k の値を指定します。

In [ ]:
cv = 5

ランダムにハイパーパラメータが抽出されるため、再現性の確保のために乱数のシードの固定を行います。

In [ ]:
# 乱数のシードの固定
random_state = 0

定義した値を用いてモデルの定義を行います。

In [ ]:
# モデルの定義
tuned_model = RandomizedSearchCV(estimator=estimator, param_distributions=param_distributions, n_iter=n_iter, cv=cv, random_state=random_state, return_train_score=False)

In [ ]:
# モデルの学習＆検証
tuned_model.fit(x_train_val, t_train_val)

今回試行回数を 100 回に設定しているため、学習結果を検証用データセットに対しての順位を表す `rank_test_score` の値を基準に昇順に並び替えて表示します。

In [ ]:
# 学習結果の確認（スコアの高い順に表示）
pd.DataFrame(tuned_model.cv_results_).sort_values('rank_test_score').T

`params` の値を確認しましょう。  
それぞれのハイパーパラメータがランダムに組み合わせられていることが確認できます。  

最も検証用データセットに対しての予測精度が高かったモデルを取得し、テスト用データセットに対しての予測精度を確認しましょう。  

In [ ]:
# 最も予測精度の高かったハイパーパラメータの確認
tuned_model.best_params_

In [ ]:
# 最も予測精度の高かったモデルの引き継ぎ
model = tuned_model.best_estimator_

In [ ]:
# モデルの検証
print(model.score(x_train_val, t_train_val))
print(model.score(x_test, t_test))

ランダムサーチは前述の通り、指定したハイパーパラメータを網羅していないので完全とは言えないですが、どこに予測精度が高くなるハイパーパラメータがあるのかあたりをつける目的では非常に有用です。  

ランダムサーチで大体のいい予測精度に繋がるハイパーパラメータのあたりをつけ、グリッドサーチを用いてより詳細な探索を行うという方法もよく用いられる方法の 1 つになります。それぞれのハイパーパラメータの調整方法には長所と短所があることを理解しておきましょう。

### ベイズ最適化
最後にベイズ最適化についてお伝えします。  

ベイズ最適化では、事前分布と事後分布と呼ばれる確率統計の理論を使用してハイパーパラメータの探索を行います。  
その際、**探索**と**活用**と呼ばれる試行錯誤を繰り返します。イメージとしては人間が行う試行錯誤に近いものがあります。  
ハイパーパラメータ探索を手動で行う際、まず初めに適当な値を入れるでしょう。そして、もう一度適当な値を入れて 1 度目の予測精度と比較し、次の探索する場所を決めていきます。このように未知の領域に対して適当に値を当てはめることを探索、探索により得た情報を元にハイパーパラメータを設定することを活用と呼びます。  

探索と活用をまとめると下記のように表現することができます。  

- 探索：まだ試していない値の範囲でハイパーパラメータを更新して、予測精度がどう変化するか情報を得る。  
- 活用：探索で得られた情報をもとに、予測精度が高まる可能性が高い範囲にハイパーパラメータを更新する。  

この手法は数学的背景の理解が難しいため、厳密な説明は省略します。
詳細は[こちら](https://en.wikipedia.org/wiki/Bayesian_optimization)を参照してください。  

ランダムサーチでは、ランダムにハイパーパラメータの値を抽出し学習を行いましたが、ベイズ最適化では探索や活用で得られた情報を元にハイパーパラメータを調整していくため、より効率的に予測精度が高くなるハイパーパラメータを見つけることができると言われています。  

ベイズ最適化を実装するためには **Optuna** というフレームワークを使用します。Optuna に関しての詳細はこちらの[公式ページ](https://optuna.org/)を参照してください。  
実装時のオプションの詳細などに関してはこちらの[公式ドキュメント](https://optuna.readthedocs.io/en/latest/index.html)を確認してください。  

Colab には Optuna はインストールされていないため、下記のコマンドを実行してインストールを行います。その他のパッケージも基本的には下記のように `pip install パッケージ名` でインストールできることも覚えておきましょう。

In [ ]:
# optuna のインストール
!pip install -q optuna

In [ ]:
import optuna

`optuna` では最初に関数 `objective` を定義して内部に以下の要素を定義します。    
- ① ハイパーパラメータごとに探索範囲を指定
- ② 学習に使用するアルゴリズムを指定
- ③ 学習の実行、検証結果の表示

探索範囲の指定にはデフォルトで準備されている `trial` クラスを使用します。      
目的に応じて設定方法が異なるので、詳細は[公式ドキュメント](https://optuna.readthedocs.io/en/latest/reference/trial.html#)を参照してください。  

③では学習・検証を繰り返してハイパーパラメータの調整を行うのですが、その際に `return` で取得した検証結果を最小化するように調整が進みます。そのため、 `return` で返す値は`1 - accuracy` （ 1 - 予測精度 = 誤分類した割合）とします。  

また、③でk-分割交差検証を使用するには `cross_validate` が必要である点も認識しておきましょう。  

In [ ]:
from sklearn.model_selection import cross_val_score

def objective(trial, x, t, cv):  
  #  ① ハイパーパラメータごとに探索範囲を指定
  max_depth = trial.suggest_int('max_depth', 2, 100)
  min_samples_split = trial.suggest_int('min_samples_split', 2, 100)
  
  # ② 学習に使用するアルゴリズムを指定
  estimator = DecisionTreeClassifier(
      max_depth = max_depth, 
      min_samples_split = min_samples_split
  )
  
  # ③ 学習の実行、検証結果の表示
  print('Current_params : ', trial.params)
  accuracy = cross_val_score(estimator, x, t, cv=cv).mean()
  return 1 - accuracy

準備が整ったら `study.optimize` を定義・実行してハイパーパラメータの調整を行います。  
`lambda` を使用して `objective` 関数に追加の引数を渡す点と、 `n_trials` で試行回数を指定する点を抑えておきましょう。    
※詳細は[公式ドキュメント](https://optuna.readthedocs.io/en/latest/faq.html)を確認してください。



In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(0)) # シードの固定

cv = 5
study.optimize(lambda trial: objective(trial, x_train_val, t_train_val, cv), n_trials=10)
print(study.best_trial)

`print` で出力している値はハイパーパラメータの値になります。  
学習が完了するたびに、現在の 1 - 正解率を表す `resulted in value` と現在までの最も良かった 1 - 正解率 を表示しています。  

学習が終了したので、最も予測精度の高かったハイパーパラメータを確認するために `study.best_params` を実行します。  

In [ ]:
# 最も予測精度の高かったハイパーパラメータの確認
study.best_params

Optuna でのハイパーパラメータ調整は先ほどと異なり、最も予測精度の高かったハイパーパラメータのみが取得でき、学習済みモデルは取得することができないため、再度学習を行う必要があります。  

下記のように `**` のようにアスタリスクを 2 つ付け、先程のハイパーパラメータをモデルのインスタンス化を行う際に引数に渡すことで、ハイパーパラメータを設定することができます。

In [ ]:
# 最適なハイパーパラメータを設定したモデルの定義
model = DecisionTreeClassifier(**study.best_params)

In [ ]:
# モデルの学習
model.fit(x_train_val, t_train_val)

In [ ]:
# モデルの検証
print(model.score(x_train_val, t_train_val))
print(model.score(x_test, t_test))

ベイズ最適化を用いてもハイパーパラメータ調整が行うことができました。  
それぞれの手法を引き出しとしてもち、それぞれの長所・短所を踏まえた上で手法を選択できるようにしましょう。  

## 練習問題 本章のまとめ

本章で学んだ内容を復習しましょう。下記の内容を次のセルに記述し、実行結果を確認してください。（必要に応じてセルの追加を行ってください。）  

- コードセルを実行し、データセットを読み込み入力変数 `x` と目標値 `t` の取得
- 訓練用データセット＆検証用データセット (`x_train_val`, `t_train_val`) とテスト用データセット (`x_test`, `t_test`) に分割（テストデータの割合 : 20% 、random_state : 0 ）
- `GridSearchCV` クラスを用いて学習＆検証を行うために必要な値の定義
    - 学習に使用するアルゴリズム : 決定木
    - 探索するハイパーパラメータの範囲
    - k-分割交差検証の k の値 ： 5
- 上記で定義した値を用い、`GridSearchCV` クラスをインスタンス化
- モデルの学習、検証
- 検証結果の確認

*ヒント*  
探索するハイパーパラメータとその範囲は自分で設定してみましょう。  
また、一度学習と検証を実行し、その検証結果から更にハイパーパラメータの範囲を調整し、もう一度探索する範囲を限定して、学習と検証を実行してみましょう。    

In [ ]:
# データセットの読み込み
from sklearn.datasets import load_iris
dataset = load_iris()
x = dataset.data
t = dataset.target
columns = dataset.feature_names
df = pd.DataFrame(x, columns=columns)
df['Target'] = t

In [ ]:
# データセットの確認
df.head(3)

In [ ]:
# 訓練用データセット＆検証用データセットとテスト用データセットに分割（テストデータの割合 : 20% 、random_state : 0）
from sklearn.model_selection import train_test_split
x_train_val, x_test, t_train_val, t_test = train_test_split(x, t, test_size=0.2, random_state=0)

In [ ]:
# 学習に使用するアルゴリズムの定義
from sklearn.tree import DecisionTreeClassifier
estimator = DecisionTreeClassifier(random_state=0)

In [ ]:
# ハイパーパラメータを探索する範囲の定義


In [ ]:
# k-分割交差検証の k の値の定義


In [ ]:
# GridSearchCV クラスを用いたモデルの定義


In [ ]:
# モデルの学習＆検証


In [ ]:
# 検証結果の確認


<img src="http://drive.google.com/uc?export=view&id=1g2xjXbw5qYeqdJqcOf3uASvzBQxhlE8u" width=30%>